In [170]:
import pandas as pd

ds = pd.read_csv("../data/small_dataset.txt")
ds.iloc[:5000].to_csv("../data/smaller_dataset.txt",index=False)

In [8]:
# Make necessary imports
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.mllib.random import RandomRDDs
from pyspark.sql import SQLContext
from pyspark import sql
import numpy as np

In [2]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
sqlContext = sql.SQLContext(sc) #?

22/03/05 15:08:03 WARN Utils: Your hostname, Kamiels-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 129.104.246.162 instead (on interface en0)
22/03/05 15:08:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/05 15:08:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/05 15:08:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/05 15:08:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
/Users/kamielfokkink/opt/anaconda3/envs/Databases/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warning

In [2]:
def normalization(x):
    s = sum(x[1])
    s_last = 0
    x_new = []
    for i in range(len(x[1])-1):
        x_new.append(x[1][i] / s)
        s_last += x_new[i]
    x_new.append(1 - s_last)
    return (x[0], np.array(x_new))

def create_uniform_rdd_rowindexed_matrix(sc, nrow, ncol, normalize=False):
    rdd = []
    for i in range(nrow):
        rdd.append((i, np.random.rand(ncol)))
    #print(rdd)
    rdd = sc.parallelize(rdd)
    if normalize:
        rdd = rdd.map(normalization)
    return rdd

In [4]:
H = create_uniform_rdd_rowindexed_matrix(sc, 4, 4, normalize=True)
H.collect()

[(0, array([0.34860894, 0.16783557, 0.35134763, 0.13220786])),
 (1, array([0.10822922, 0.3555388 , 0.32625561, 0.20997637])),
 (2, array([0.02984418, 0.29335671, 0.33616264, 0.34063647])),
 (3, array([0.0272727 , 0.07928486, 0.63217075, 0.2612717 ]))]

In [87]:
def update_h(hxy_tuple):
    hx, y = hxy_tuple
    h, x = hx
    d = h * y 
    e = h * x
    h_new = h * np.divide(x+d, y+e)
    return h_new

In [168]:
class MRAlgorithm:
    
    def __init__(self, datapath, k):
        self.k = k
        self.n = 2649429
        self.m = 6
        self.sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]").set("spark.driver.memory", "15g"))
        self.input = self.sc.textFile(datapath).map(lambda x: (int(x.split(',')[0])-1,
                                                               int(x.split(',')[1])-1,int(x.split(',')[2])))
        self.A = self.input
        self.H = create_uniform_rdd_rowindexed_matrix(self.sc, self.n, self.k, normalize=True)
        self.W = create_uniform_rdd_rowindexed_matrix(self.sc, self.m, self.k)
        
    def compute_H(self):
        print("Compute H")
    
        ## compute X
        # M1
        W_b = self.sc.broadcast(self.W.collect()) # a dictionary where the key is the index i
        Aw = self.A.map(lambda x: (x[1], x[2]*W_b.value[x[0]][1])) #Wi.T
        #print(Aw.collect())
        # R1 
        X = Aw.reduceByKey(lambda x, y: x+y) # add a combiner? #.map(lambda x: (x[0], x[1]))
    
        
        ## compute B
        # M2
        ww = self.W.map(lambda x: (None, np.outer(x[1],x[1]))) # w.T.dot(w)? #import numpy.linalg
        # from pyspark.mllib.linalg.distributed import *
        # as_block_matrix(w.T).multiply(as_block_matrix(w))
        # https://stackoverflow.com/questions/37766213/spark-matrix-multiplication-with-python
        # R2
        B = ww.reduceByKey(lambda x, y: x+y).map(lambda x: x[1]) # without map?
        
        ## compute Y
        # M3
        B_b = self.sc.broadcast(B.collect())
        y = self.H.map(lambda x: (x[0], B_b.value@x[1]))
        
        ## update H
        # M4
        self.hxj = self.H.join(y).join(X)
        # R4
        # why is it a reduce phase instead of a map phase?
        hnew = self.hxj.map(lambda x: (x[0], update_h(x[1])))
        self.H = hnew
        
    def compute_W(self):
        print("Compute W")
        #M5: Compute matrix S (repartition join of H and A)
        tmp_M5 = self.H.join(self.A.map(lambda x: (x[1], (x[0], x[2]))))

        #R5: Emit each hj according to row i
        self.tmp_M5 = tmp_M5.map(lambda x: (x[1][1][0], x[1][1][1] * x[1][0]))

        
        
        #M6: Finish the computation of matrix S -- identity mapper so no code needed
        #R6: Emit each row in matrix S
        self.S = self.tmp_M5.reduceByKey(lambda x, y : x +y)

        
        #M7: Calculate matrix C (same as calculating B in compute_H)
        tmp_M7 = self.H.map(lambda x: (0, np.outer(x[1], x[1])))
        #R7: Emit matrix C from the sum of matrices
        C = tmp_M7.reduceByKey(lambda x, y : x +y).map(lambda x: x[1])
        
        #M8: Compute matrix T
        C_b = self.sc.broadcast(C.collect())
        self.T = self.W.map(lambda x: (x[0], x[1]@C_b.value))
        

        #M9: update W with broadcast join of S and T. W = W * S/T
        T_b = self.sc.broadcast(self.T.collectAsMap())
        S_b = self.sc.broadcast(self.S.collectAsMap())
        self.W = self.W.map(lambda x: (x[0], x[1]*np.divide(S_b.value[x[0]], T_b.value[x[0]])))

        
    def assign_clusters(self):
        #M10: Map user to cluster with highest probability
        #self.clusters = self.H.map(lambda x: (x[0], x[1].index(max(x[1]))))
        self.clusters = self.H.map(lambda x: (x[0], np.where(x[1][0] == max(x[1][0]))[0][0]))
        
        #M11: Emit a 1 for each user that is in a cluster
        self.clustersizes = self.clusters.map(lambda x: (x[1], 1))
        
        #R11: Count the number of users per cluster
        self.clustersizes = self.clustersizes.reduceByKey(lambda x,y: x+y)
        
    def RM2_distribution(self):
        #M12: Emit each rating that a user gave
        self.ratings = self.input.map(lambda x: (x[1], x[2]))
        
        #R12: Sum ratings given by user
        self.ratings = self.ratings.reduceByKey(lambda x, y: x+y)
        
        #Accumulator containing total count of ratings
        globalcount = self.sc.accumulator(0)
        self.ratings.foreach(lambda x: globalcount.add(x[1]))
        
        #M13: Emit each rating that an item received
        self.items = self.input.map(lambda x: (x[0], x[2]))
        
        #Broadcast the total count of ratings, to be used in next reduce
        globalcount = self.sc.broadcast(globalcount.value)
        print("Globalcount is: ", globalcount.value)
        
        #R13: Compute the probability of item in the collection
        self.items = self.items.reduceByKey(lambda x,y: x+y)
        
        #Divide previous values by the total count of ratings, to normalize probabilities
        self.items = self.items.map(lambda x: (x[0], x[1]/globalcount.value))
        
        #Broadcast all users and which cluster they are in
        clusters = self.sc.broadcast(self.clusters.collectAsMap())
        
        #M14-a: Map each rating from input to the cluster assigned to user
        self.input2cluster = self.input.map(lambda x: (clusters.value[x[1]], (x[1], x[0], x[2])))
        
        #M14-b: Map sum of ratings of a user to their assigned cluster
        self.ratings2cluster = self.ratings.map(lambda x: (clusters.value[x[0]], (x[0], x[1])))
        
        #Repartition join these two maps by cluster
        self.fullClusters = self.input2cluster.join(self.ratings2cluster)
        
        #Broadcast sizes of the clusters
        clustersizes = self.sc.broadcast(self.clustersizes.collectAsMap())
        
        #Broadcast probabilities of each item
        items = self.sc.broadcast(self.items.collectAsMap())
        
        #Final reduce combines all information from above, needs W and H

In [169]:
#Algorithm = MRAlgorithm('../data/small_dataset.txt', k=5)
Algorithm = MRAlgorithm('../data/smaller_dataset.txt', k=5)
Algorithm.compute_H()
Algorithm.compute_W()
Algorithm.assign_clusters()
Algorithm.RM2_distribution()

Compute H
Compute W


22/03/05 17:56:27 WARN TaskSetManager: Stage 465 contains a task of very large size (23171 KiB). The maximum recommended task size is 1000 KiB.


Globalcount is:  17885


In [51]:
Algorithm.input.take(10)

[(1, 1488844, 3),
 (1, 822109, 5),
 (1, 885013, 4),
 (1, 30878, 4),
 (1, 823519, 3),
 (1, 893988, 3),
 (1, 124105, 4),
 (1, 1248029, 3),
 (1, 1842128, 4),
 (1, 2238063, 3)]

In [162]:
Algorithm.clusters.collectAsMap()

{145872: 4,
 590112: 4,
 664704: 3,
 720144: 3,
 871488: 1,
 895968: 4,
 963360: 1,
 1063872: 1,
 1069488: 3,
 1088784: 2,
 1346256: 0,
 1560816: 0,
 1581264: 2,
 1587024: 0,
 1648656: 2,
 1662624: 0,
 1716192: 2,
 1763856: 3,
 1870128: 3,
 1935792: 0,
 1984464: 3,
 2086128: 1,
 2185056: 2,
 2237184: 1,
 2468304: 2,
 2490048: 3,
 2541024: 1,
 2580480: 2,
 78930: 1,
 154818: 4,
 341586: 3,
 396594: 1,
 453042: 3,
 527490: 2,
 592002: 2,
 599634: 3,
 679554: 2,
 866610: 4,
 900306: 1,
 926514: 3,
 952146: 4,
 1094850: 2,
 1160370: 1,
 1176642: 3,
 1342386: 3,
 1346994: 2,
 1370322: 0,
 1373922: 1,
 1445346: 3,
 1479906: 2,
 1495746: 4,
 1562706: 1,
 1616562: 1,
 1877346: 3,
 1927890: 3,
 1928178: 1,
 1998162: 3,
 2070018: 2,
 2314530: 2,
 2317410: 4,
 2344482: 3,
 2352258: 3,
 2513538: 4,
 2563362: 3,
 1332: 0,
 26100: 1,
 161892: 3,
 219924: 2,
 250596: 2,
 360756: 3,
 371700: 2,
 521460: 3,
 538740: 1,
 552996: 1,
 686628: 3,
 688356: 1,
 746964: 0,
 791028: 2,
 944244: 2,
 1001700: 4,

In [163]:
Algorithm.clustersizes.collect()

[(0, 831), (1, 1134), (2, 866), (3, 1367), (4, 632)]

In [164]:
Algorithm.ratings.collect()

[(822108, 5),
 (885012, 9),
 (823518, 3),
 (124104, 4),
 (1248028, 3),
 (2238062, 3),
 (1503894, 4),
 (2590060, 3),
 (543864, 4),
 (1209118, 4),
 (804918, 4),
 (1086806, 3),
 (1711858, 4),
 (372232, 5),
 (1080360, 3),
 (814700, 5),
 (808730, 4),
 (337540, 5),
 (1537426, 4),
 (2381598, 3),
 (1910568, 4),
 (2421814, 2),
 (1481960, 2),
 (401046, 5),
 (2179072, 3),
 (2647870, 9),
 (1905580, 5),
 (2508818, 3),
 (1578278, 1),
 (1159694, 4),
 (2423090, 3),
 (2148698, 2),
 (1342006, 3),
 (466134, 4),
 (1546548, 5),
 (1493696, 1),
 (494608, 4),
 (1961618, 5),
 (573536, 4),
 (411704, 4),
 (314932, 3),
 (1792740, 3),
 (769642, 1),
 (1696030, 4),
 (1817214, 4),
 (406056, 4),
 (593224, 3),
 (2630336, 5),
 (1155746, 3),
 (2439492, 6),
 (530788, 5),
 (231000, 3),
 (1493614, 5),
 (1850614, 2),
 (68958, 3),
 (624034, 5),
 (1158758, 4),
 (1215396, 5),
 (2475250, 4),
 (321110, 2),
 (2635436, 4),
 (2389366, 1),
 (235552, 4),
 (831868, 4),
 (1871178, 3),
 (2040858, 1),
 (2287002, 5),
 (1346256, 3),
 (30244

In [165]:
Algorithm.items.collect()

[(0, 0.11467710371819961),
 (2, 0.40961699748392505),
 (4, 0.24981828347777468),
 (1, 0.02885099245177523),
 (3, 0.021750069890970086),
 (5, 0.17528655297735532)]

In [171]:
Algorithm.fullClusters.collect()

[(0, ((823518, 0, 3), (823518, 3))),
 (0, ((823518, 0, 3), (124104, 4))),
 (0, ((823518, 0, 3), (804918, 4))),
 (0, ((823518, 0, 3), (814700, 5))),
 (0, ((823518, 0, 3), (337540, 5))),
 (0, ((823518, 0, 3), (1910568, 4))),
 (0, ((823518, 0, 3), (2179072, 3))),
 (0, ((823518, 0, 3), (2508818, 3))),
 (0, ((823518, 0, 3), (1578278, 1))),
 (0, ((823518, 0, 3), (2148698, 2))),
 (0, ((823518, 0, 3), (1493696, 1))),
 (0, ((823518, 0, 3), (1961618, 5))),
 (0, ((823518, 0, 3), (406056, 4))),
 (0, ((823518, 0, 3), (2630336, 5))),
 (0, ((823518, 0, 3), (1493614, 5))),
 (0, ((823518, 0, 3), (1850614, 2))),
 (0, ((823518, 0, 3), (235552, 4))),
 (0, ((823518, 0, 3), (1346256, 3))),
 (0, ((823518, 0, 3), (1904904, 4))),
 (0, ((823518, 0, 3), (493008, 7))),
 (0, ((823518, 0, 3), (850326, 5))),
 (0, ((823518, 0, 3), (2329564, 4))),
 (0, ((823518, 0, 3), (255442, 2))),
 (0, ((823518, 0, 3), (2315072, 4))),
 (0, ((823518, 0, 3), (2333816, 3))),
 (0, ((823518, 0, 3), (2090476, 4))),
 (0, ((823518, 0, 3), 

In [172]:
Algorithm.ratings2cluster.collect()

[(2, (822108, 5)),
 (2, (885012, 9)),
 (0, (823518, 3)),
 (0, (124104, 4)),
 (4, (1248028, 3)),
 (2, (2238062, 3)),
 (2, (1503894, 4)),
 (3, (2590060, 3)),
 (1, (543864, 4)),
 (2, (1209118, 4)),
 (0, (804918, 4)),
 (1, (1086806, 3)),
 (2, (1711858, 4)),
 (1, (372232, 5)),
 (4, (1080360, 3)),
 (0, (814700, 5)),
 (1, (808730, 4)),
 (0, (337540, 5)),
 (2, (1537426, 4)),
 (2, (2381598, 3)),
 (0, (1910568, 4)),
 (2, (2421814, 2)),
 (1, (1481960, 2)),
 (3, (401046, 5)),
 (0, (2179072, 3)),
 (3, (2647870, 9)),
 (2, (1905580, 5)),
 (0, (2508818, 3)),
 (0, (1578278, 1)),
 (1, (1159694, 4)),
 (2, (2423090, 3)),
 (0, (2148698, 2)),
 (4, (1342006, 3)),
 (4, (466134, 4)),
 (2, (1546548, 5)),
 (0, (1493696, 1)),
 (4, (494608, 4)),
 (0, (1961618, 5)),
 (2, (573536, 4)),
 (1, (411704, 4)),
 (2, (314932, 3)),
 (2, (1792740, 3)),
 (2, (769642, 1)),
 (2, (1696030, 4)),
 (2, (1817214, 4)),
 (0, (406056, 4)),
 (3, (593224, 3)),
 (0, (2630336, 5)),
 (2, (1155746, 3)),
 (2, (2439492, 6)),
 (1, (530788, 5)),


In [108]:
Algorithm.W.collect()

[(0, array([[0.00927442, 0.01022214, 0.02123217, 0.02689467, 0.01834447]])),
 (1, array([[0.00538309, 0.00416177, 0.00404351, 0.0036074 , 0.00267794]])),
 (2, array([[0.07031543, 0.04285017, 0.06022516, 0.02656751, 0.06703434]])),
 (3, array([[0.00284574, 0.0050023 , 0.00498435, 0.00132596, 0.00191443]])),
 (4, array([[0.03303835, 0.04705092, 0.04352636, 0.01998488, 0.05845106]])),
 (5, array([[0.0219204 , 0.03590892, 0.01212355, 0.03987945, 0.02336788]])),
 (6, array([[0.00212754, 0.0011059 , 0.00245932, 0.00062962, 0.00201476]])),
 (7, array([[0.37604632, 0.2487154 , 0.41289369, 0.45384713, 0.35387226]])),
 (8, array([[0.00230283, 0.00088212, 0.00213813, 0.00283449, 0.00188393]])),
 (9, array([[0.00730612, 0.00478311, 0.00587988, 0.00843109, 0.0042601 ]])),
 (10, array([[0.00554621, 0.00480717, 0.00363777, 0.00468457, 0.00362716]])),
 (11, array([[0.01543845, 0.01384189, 0.00715284, 0.01842715, 0.01322484]])),
 (12, array([[0.00403611, 0.00340085, 0.00333105, 0.00334291, 0.00342564]]

In [109]:
Algorithm.H.collect()

[(4752, array([[0.43591318, 0.17733824, 0.39537767, 0.09548032, 0.40675621]])),
 (8208, array([[0.51479314, 0.60247823, 0.20862756, 0.4124752 , 0.4303087 ]])),
 (16272,
  array([[0.32987645, 0.18520456, 0.52887532, 0.63427179, 0.93869326]])),
 (17712,
  array([[0.05177027, 0.53406607, 0.40013359, 0.2186335 , 0.52403387]])),
 (18720,
  array([[0.45269671, 0.05583543, 0.21114931, 0.47881789, 0.46964473]])),
 (26208,
  array([[0.26023951, 0.70346207, 0.15897246, 0.26441837, 0.39358311]])),
 (28656,
  array([[0.5265604 , 0.30264305, 0.49602136, 0.46370299, 0.26168454]])),
 (29664,
  array([[0.14745852, 0.37031999, 0.64053303, 0.75054939, 0.27931823]])),
 (31968,
  array([[0.48077255, 0.57365062, 0.24210574, 0.06696011, 0.44819087]])),
 (52416,
  array([[0.84006657, 0.87490215, 0.20400302, 0.17498068, 0.58887373]])),
 (79632,
  array([[0.147156  , 0.86331143, 0.72728272, 0.24641255, 0.78788074]])),
 (93024,
  array([[0.84091217, 0.15930857, 0.59218324, 0.3734644 , 0.27040256]])),
 (116352,
